In [157]:
import numpy as np
import pandas as pd
import pathlib
import os
from decimal import *
getcontext().prec = 5

df_probabilities = {
	6: pd.read_csv('vars\\prob_06.csv', header=None),
	12: pd.read_csv('vars\\prob_12.csv', header=None)
}

df_tables = {
	6: pd.read_csv('vars\\table_06.csv', header=None),
	12: pd.read_csv('vars\\table_12.csv', header=None)
}

lengths = {
	6: {
		'text_count': len(df_tables[6].columns),
		'key_count': len(df_tables[6])
	},
	12: {
		'text_count': len(df_tables[12].columns),
		'key_count': len(df_tables[12])
	}
}


In [158]:
def get_cipher_text_unconditional_probability(variant, cipher_text):
	df_table = df_tables[variant]
	df_prob = df_probabilities[variant]
	
	probability = 0.0

	for plain_text_index in range(lengths[variant]['text_count']):
		for key_text_index in range(lengths[variant]['key_count']):
			if (df_table[plain_text_index][key_text_index] == cipher_text):
				plain_text_probability = df_prob[plain_text_index][0]
				key_probability = df_prob[key_text_index][1]

				probability += plain_text_probability * key_probability
	
	return probability

df_cipher_texts_unconditional_prob = {
	6: np.array([get_cipher_text_unconditional_probability(6, i) for i in range(lengths[6]['text_count'])]),
	12: np.array([get_cipher_text_unconditional_probability(12, i) for i in range(lengths[12]['text_count'])])
}


In [159]:
def cipher_and_plain_texts_common_probability(variant, cipher_text, plain_text):
	df_table = df_tables[variant]
	df_prob = df_probabilities[variant]
	
	probability = 0.0

	for key_index in range(lengths[variant]['key_count']):
		if (df_table[plain_text][key_index] == cipher_text):
			plain_text_probability = df_prob[plain_text][0]
			key_probability = df_prob[key_index][1]

			probability += plain_text_probability * key_probability
	
	return probability

df_cipher_texts_common_prob = {
	6: np.array([[cipher_and_plain_texts_common_probability(6, i, j) for j in range(lengths[6]['text_count'])] for i in range(lengths[6]['text_count'])]),
	12: np.array([[cipher_and_plain_texts_common_probability(12, i, j) for j in range(lengths[12]['text_count'])] for i in range(lengths[12]['text_count'])])
}

# sum([pd.DataFrame(df_cipher_texts_common_prob[6][i]).sum() for i in range(20)])


In [160]:
def cipher_and_plain_texts_conditional_probability(variant):
    return np.array([
			[df_cipher_texts_common_prob[variant][i][j] / df_cipher_texts_unconditional_prob[variant][i] for j in range(lengths[6]['text_count'])
		] for i in range(lengths[6]['text_count'])])

df_cipher_texts_conditional_prob = {
	6: cipher_and_plain_texts_conditional_probability(6),
	12: cipher_and_plain_texts_conditional_probability(12)
}


In [161]:
def get_bayes_decision_function(variant, cipher_text):
	return np.argmax(df_cipher_texts_conditional_prob[variant][cipher_text])

df_bayes_decision_function = {
	6: [get_bayes_decision_function(6, i) for i in range(lengths[6]['text_count'])],
	12: [get_bayes_decision_function(12, i) for i in range(lengths[6]['text_count'])]
}


In [162]:
def get_bayes_loss_function(variant, cipher_text, plain_text):
	return 0 if df_bayes_decision_function[6][cipher_text] == plain_text else 1

def get_average_loss_function(variant):
	return sum(
		[get_bayes_loss_function(variant, i, j) * df_cipher_texts_common_prob[variant][i][j] for j in range(lengths[6]['text_count']) for i in range(lengths[6]['text_count'])]
	)

df_bayes_loss_function = {
	6: get_average_loss_function(6),
	12: get_average_loss_function(12)
}


0.9249999999999963

In [163]:
def get_stochastic_decision_function(variant):
	matrix = np.zeros((lengths[variant]['text_count'], lengths[variant]['text_count']))

	for i in range(lengths[variant]['text_count']):
		current_probability_set = df_cipher_texts_conditional_prob[variant][i]

		max_probability = max(current_probability_set)
		max_probability_entrance = list(current_probability_set).count(max_probability)

		for j in range(lengths[variant]['text_count']):
			matrix[i][j] = 1 / max_probability_entrance if current_probability_set[j] == max_probability else 0

	return matrix

df_stochastic_decision_function = {
	6: [get_stochastic_decision_function(6)],
	12: [get_stochastic_decision_function(12)]
}
